# 프로젝트 - Movielens 영화 추천 실습

활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

* 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
* 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
* 별점을 시청횟수로 해석해서 생각하겠습니다.
* 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.


In [182]:
# import library
import numpy as np
import scipy
import implicit

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


## 1) 데이터 준비와 전처리


In [184]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [185]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [186]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [187]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [188]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 2) 분석
* ratings에 있는 유니크한 영화 개수
* ratings에 있는 유니크한 사용자 수
* 가장 인기 있는 영화 30개(인기순)

In [189]:
# ratings에 있는 유니크한 영화 개수
num_movie = ratings['movie_id'].nunique()
num_movie

3628

In [190]:
# ratings에 있는 유니크한 사용자 수
num_user = ratings['user_id'].nunique()
num_user

6039

In [191]:
# 가장 인기 있는 영화 30개(인기순)
movie_popular = ratings.groupby('movie_id')['user_id'].count()
k = movie_popular.sort_values(ascending=False).head(30)
pd.merge(k, movies, on='movie_id')

,movie_id,user_id,title,genre
0,2858,3211,American Beauty (1999),Comedy|Drama
1,260,2910,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
2,1196,2885,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
3,1210,2716,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
4,2028,2561,Saving Private Ryan (1998),Action|Drama|War
5,589,2509,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
6,593,2498,"Silence of the Lambs, The (1991)",Drama|Thriller
7,1198,2473,Raiders of the Lost Ark (1981),Action|Adventure
8,1270,2460,Back to the Future (1985),Comedy|Sci-Fi
9,2571,2434,"Matrix, The (1999)",Action|Sci-Fi|Thriller


## 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가

In [192]:
ratings = ratings.drop('timestamp',axis=1)
ratings

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [193]:
# 마지막 유저 id
ratings['user_id'].max()

6040

In [194]:
# 나의 유저 id (= 6041) 와 선호 영화 id 추가
my1 = {'user_id': 6041, 'movie_id': 260, 'counts': 5}
my2 = {'user_id': 6041, 'movie_id': 1196, 'counts': 5}
my3 = {'user_id': 6041, 'movie_id': 1210, 'counts': 5}
my4 = {'user_id': 6041, 'movie_id': 589, 'counts': 5}
my5 = {'user_id': 6041, 'movie_id': 480, 'counts': 5}
ratings = ratings.append(my1, ignore_index = True)
ratings = ratings.append(my2, ignore_index = True)
ratings = ratings.append(my3, ignore_index = True) 
ratings = ratings.append(my4, ignore_index = True) 
ratings = ratings.append(my5, ignore_index = True) 

In [195]:
ratings[836470:836484]

,user_id,movie_id,counts
836470,6040,2028,5
836471,6040,1080,4
836472,6040,1089,4
836473,6040,1090,3
836474,6040,1094,5
836475,6040,562,5
836476,6040,1096,4
836477,6040,1097,4
836478,6041,260,5
836479,6041,1196,5


## 4) CSR matrix를 직접 만들기


In [196]:
# 고유한 유저, 영화 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [197]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx[6041])
print(movie_to_idx[1])# 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 

6039
40


In [198]:
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

user_id column indexing OK!!


In [199]:
# movie_to_idx을 통해 movie_id 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

movie_id column indexing OK!!


In [200]:
ratings

,user_id,movie_id,counts
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
836478,6039,44,5
836479,6039,117,5
836480,6039,64,5
836481,6039,92,5


In [201]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape = (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성 & 훈련

In [202]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [203]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [204]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [205]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도

In [206]:
me, Matirx = user_to_idx[6041], movie_to_idx[2571]
my_vector, matrix_vector = als_model.user_factors[me], als_model.item_factors[Matirx]

print('슝=3')

슝=3


In [207]:
my_vector

array([ 0.64878273,  0.26153785, -0.54369724, -0.32430395, -0.4991337 ,
       -1.0095179 , -0.31403047,  0.1964158 , -0.5377145 , -0.56438905,
        0.21830027,  0.9616008 ,  0.34243643,  0.42727882,  0.15734404,
        0.51325274,  0.43599898,  0.41063014,  0.57902396,  0.37631744,
        0.24783802,  0.31786957, -0.6530617 ,  0.6072334 ,  0.06365278,
        0.15030323,  0.9654379 ,  0.03209203,  0.4579913 ,  0.9625484 ,
       -0.21380845, -1.0901873 , -0.36529556, -0.06237607,  0.6842617 ,
       -0.2177949 , -0.25962615,  0.31202206,  0.89020616,  0.6980329 ,
       -0.00590576, -0.98783135, -0.1796805 ,  0.69346005,  0.20112474,
        0.17644863, -0.08770169, -0.23124948, -1.2112929 ,  0.7966639 ,
        0.36689827,  0.35882583,  0.186021  ,  0.44452807,  0.07058833,
       -1.1233401 , -0.14129636,  0.27053338,  0.69420904, -0.6052003 ,
       -1.0040946 , -0.25867033, -0.36412227, -0.61394465,  0.61867225,
        0.7981805 ,  1.0571021 , -0.33286664, -0.45970663, -0.19

In [208]:
matrix_vector

array([ 0.03144126,  0.02240729, -0.01216483,  0.00793418,  0.02810876,
       -0.00279822,  0.01975317,  0.01849595, -0.03336544,  0.00185772,
        0.00833815,  0.0256633 ,  0.0145984 ,  0.01854519,  0.01662116,
       -0.00075018,  0.01026541,  0.037894  ,  0.01859338,  0.00303502,
        0.00367564, -0.00348339, -0.00208284,  0.03336024,  0.03458932,
       -0.01571877,  0.0246808 , -0.00053558,  0.00551901,  0.01660543,
        0.01969987, -0.03436152, -0.01122528, -0.00434091, -0.00684314,
       -0.01075633,  0.01668479,  0.02077681,  0.02605258,  0.02276645,
       -0.00337557, -0.03968243,  0.01011323,  0.0207909 ,  0.01976655,
        0.00720333,  0.01776955, -0.00633263, -0.00283437,  0.00011201,
        0.01969634,  0.00515187,  0.03686822,  0.01600665,  0.00041776,
       -0.02889264, -0.00935512,  0.00070837,  0.03072545, -0.01945682,
       -0.02416616, -0.01450357, -0.00869216,  0.00229945, -0.00298357,
        0.00031944,  0.03286518, -0.00299211, -0.01215831, -0.01

In [209]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(my_vector, matrix_vector)

0.5630018

### 분석 : 
모델의 분석으로는  내가 매트릭스 영화에 대한 선호도는 0.56으로 1에 크게 가까운 지표는 아니나, 0.5를 넘겼으니 유의미한 결과라고 생각된다.

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천

In [211]:
favorite_movie = 2 #Jumanji
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(513, 1.0000001),
 (596, 0.8067744),
 (1130, 0.80283934),
 (173, 0.7401758),
 (1982, 0.64632535),
 (828, 0.63236094),
 (1985, 0.62991744),
 (545, 0.6276787),
 (1733, 0.6247044),
 (1736, 0.6160259),
 (561, 0.5689921),
 (576, 0.5609352),
 (942, 0.5463509),
 (2017, 0.5442092),
 (2009, 0.539727)]

In [213]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
idx_similar_movie = [idx_to_movie[i[0]] for i in similar_movie]

In [229]:
title_similar_movie = [movies[movies.movie_id == i]['title'].values[0] for i in idx_similar_movie]
title_similar_movie

['Jumanji (1995)',
 'Hook (1991)',
 'Indian in the Cupboard, The (1995)',
 'Dragonheart (1996)',
 'Space Jam (1996)',
 'Flubber (1997)',
 'NeverEnding Story II: The Next Chapter, The (1990)',
 'Santa Clause, The (1994)',
 'Borrowers, The (1997)',
 'Small Soldiers (1998)',
 'NeverEnding Story, The (1984)',
 'Escape to Witch Mountain (1975)',
 'Legend (1985)',
 'Pagemaster, The (1994)',
 "Kid in King Arthur's Court, A (1995)"]

## 8) 내가 가장 좋아할 만한 영화들을 추천

In [230]:
user = user_to_idx[6041]
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(124, 0.5630018),
 (175, 0.4932823),
 (60, 0.48584616),
 (120, 0.46714938),
 (200, 0.43080476),
 (22, 0.3430649),
 (99, 0.33197528),
 (62, 0.33043373),
 (87, 0.32976693),
 (5, 0.29862714),
 (193, 0.2766662),
 (48, 0.2544483),
 (651, 0.25431442),
 (172, 0.23372658),
 (160, 0.21192375),
 (82, 0.2076147),
 (26, 0.20298782),
 (680, 0.20235115),
 (58, 0.2004929),
 (372, 0.18478237)]

In [234]:
idx_movie_recommended = [idx_to_movie[i[0]] for i in movie_recommended]
title_movie_recommended = [movies[movies.movie_id == i]['title'].values[0] for i in idx_movie_recommended]
title_movie_recommended

['Matrix, The (1999)',
 'Men in Black (1997)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Raiders of the Lost Ark (1981)',
 'Terminator, The (1984)',
 'Back to the Future (1985)',
 'American Beauty (1999)',
 'Total Recall (1990)',
 'Braveheart (1995)',
 'Princess Bride, The (1987)',
 'Alien (1979)',
 'Saving Private Ryan (1998)',
 'Aliens (1986)',
 'Indiana Jones and the Last Crusade (1989)',
 'Forrest Gump (1994)',
 'Lost World: Jurassic Park, The (1997)',
 'E.T. the Extra-Terrestrial (1982)',
 'Blade Runner (1982)',
 'Mission: Impossible (1996)',
 'X-Men (2000)']

In [238]:
Matrix = movie_to_idx[2571]
explain = als_model.explain(user, csr_data, itemid=Matrix)
explain

(0.5511566686602494,
 [(92, 0.23392770427743892),
  (107, 0.12398864142964136),
  (44, 0.07296802756604517),
  (117, 0.06559661362139801),
  (64, 0.05467568176572585)],
 (array([[ 0.62722051,  0.10587628,  0.12167134, ...,  0.09194738,
           0.08411134,  0.09252366],
         [ 0.06640777,  0.59395911,  0.08984031, ...,  0.11152048,
           0.05498305,  0.08590898],
         [ 0.07631476,  0.06624358,  0.54955946, ...,  0.08727608,
           0.09600683,  0.06669461],
         ...,
         [ 0.05767128,  0.07597365,  0.06916979, ...,  0.5339075 ,
           0.01166201,  0.03851713],
         [ 0.05275636,  0.04156308,  0.0679351 , ...,  0.07448257,
           0.52828353, -0.01034124],
         [ 0.05803274,  0.06082248,  0.05562822, ...,  0.07964015,
           0.04152347,  0.53857568]]),
  False))

In [248]:
j = [(idx_to_movie[i[0]], i[1]) for i in explain[1]]
[(movies[movies.movie_id == i[0]]['title'].values[0], i[1]) for i in j]

[('Terminator 2: Judgment Day (1991)', 0.23392770427743892),
 ('Jurassic Park (1993)', 0.12398864142964136),
 ('Star Wars: Episode IV - A New Hope (1977)', 0.07296802756604517),
 ('Star Wars: Episode V - The Empire Strikes Back (1980)',
  0.06559661362139801),
 ('Star Wars: Episode VI - Return of the Jedi (1983)', 0.05467568176572585)]

### 분석 : 
Matrix가 나의 제1순위 추천영화이다.

explain을 통해 이 추천에 기여된 정도를 확인해보면 터미네이터와 주라기 공원 모두 어드벤쳐영화에 SF 영화이기 때문에 영향을 많이 미친것을 확인할 수 있었다. 

# 회고

잘한 점 : 개념을 확실하게 이해하고 코드를 어떻게 활용하면 원하는 결과물을 얻을 수 있는지 완전히 이해하고 활용할 수 있다.

못한 점 : 코드의 수학적 모델을 시간 부족으로 공부해보지 못한 점이 아쉽다.

노력할 점 : LSA와 LDA와 관련하여 추천시스템의 수학적 원리와 모델간 차이점까지 정리할 수 있는 시간을 마련해야겠다.